# Step0 : Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

# Step3 : Download the data

### 3.1 Download the data from figshare

In [2]:
# Trigger request to download data
article_id = "14096681"
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]            

### 3.2 Extract the zip file

In [5]:
# Create zip file with downloaded data

In [4]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.58 s, sys: 11.5 s, total: 15.1 s
Wall time: 2min 10s


In [6]:
# Extract data into output directory

In [7]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.65 s, sys: 1.23 s, total: 8.88 s
Wall time: 9.29 s


# Step4 : Combining data CSVs

In [8]:
%%time
import pandas as pd
excluded_files = ["figsharerainfall\\observed_daily_rainfall_SYD.csv"]
files = glob.glob('figsharerainfall/*.csv')
files = list(set(files) - set(excluded_files))
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

CPU times: user 3min 17s, sys: 11.8 s, total: 3min 29s
Wall time: 3min 31s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Stephen    |                  |     |           |        |            |
| Nate    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |     3min 31s       |
| Natalie    |                  |     |           |        |            |
| Nikita    |    Windows              |  16GB   |   12th Gen Intel(R) Core(TM) i7-1255U | Yes |   12min 49s         |

# Step5 : Perform Simple EDA

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Stephen    |                  |     |           |        |            |
| Nate    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |            |
| Natalie    |                  |     |           |        |            |
| Nikita    |    Windows              | 16GB    |  12th Gen Intel(R) Core(TM) i7-1255U  |  yes   |            |

# Step6 : Perform Simple EDA in R

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Stephen    |                  |     |           |        |            |
| Nate    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |            |
| Natalie    |                  |     |           |        |            |
| Nikita    |    Windows              | 16GB    |  12th Gen Intel(R) Core(TM) i7-1255U  |  yes   |            |